# 🚀 Despliegue de Modelo CNN1D_LSTM en SageMaker Serverless

Este notebook te guía paso a paso para desplegar tu modelo de ECG en AWS SageMaker usando **Serverless Inference**.

## 📋 Requisitos Previos

Antes de comenzar, asegúrate de tener:

- ¢Å“"¦ Cuenta AWS activa
- ¢Å“"¦ Credenciales AWS (Access Key ID y Secret Access Key)
- ¢Å“"¦ Rol IAM con permisos de SageMaker
- ¢Å“"¦ El archivo `cnn1d_lstm_ecg_v1_sagemaker.tar.gz` en la carpeta `sagemaker_models/`

## 💰 Costos

- **Por inferencia:** ~$0.00022
- **Sin tráfico:** $0 (no hay costo cuando está inactivo)
- **Ejemplo:** 10,000 inferencias/mes = $2.20

## 📝 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Configura tus credenciales** en la celda de configuración
3. **Sigue las instrucciones** que aparecen en cada paso

---

**Tiempo estimado total:** 7-12 minutos


## 📝¦ Paso 0: Instalación de Dependencias

Ejecuta esta celda para instalar las librerías necesarias.


In [1]:
# Instalar dependencias (ejecuta solo si no las tienes instaladas)
import sys
import subprocess

def check_package_installed(package):
    """Verifica si un paquete está instalado y devuelve su versión"""
    try:
        mod = __import__(package)
        version = getattr(mod, '__version__', 'desconocida')
        return True, version
    except ImportError:
        return False, None

def install_package(package, version_constraint=None, force_reinstall=False):
    """Instala un paquete si no está instalado o si se fuerza la reinstalación"""
    is_installed, version = check_package_installed(package)
    
    if is_installed and not force_reinstall:
        print(f"¢Å“" {package} ya está instalado (versión: {version})")
        return True
    
    package_name = f"{package}{version_constraint}" if version_constraint else package
    action = "Reinstalando" if is_installed else "Instalando"
    print(f"📝¦ {action} {package_name}...")
    
    try:
        cmd = [sys.executable, "-m", "pip", "install"]
        if force_reinstall:
            cmd.append("--force-reinstall")
        cmd.append(package_name)
        
        subprocess.check_call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"¢Å“" {package} instalado correctamente")
        return True
    except Exception as e:
        print(f"¢ÂÅ’ Error instalando {package}: {e}")
        return False

# Verificar e instalar dependencias
print("=" * 70)
print("📝¦ VERIFICANDO E INSTALANDO DEPENDENCIAS")
print("=" * 70)
print("¢Å¡Â ¯Â¸Â  NOTA: Se instalará sagemaker 2.x (compatible con este notebook)")
print()

all_installed = True
sagemaker_needs_reinstall = False

# Verificar boto3
boto3_installed, boto3_version = check_package_installed('boto3')
if not boto3_installed:
    if not install_package('boto3'):
        all_installed = False
else:
    print(f"¢Å“" boto3 ya está instalado (versión: {boto3_version})")

# Verificar sagemaker
sagemaker_installed, sagemaker_version = check_package_installed('sagemaker')

if sagemaker_installed:
    print(f"¢Å“" sagemaker ya está instalado (versión: {sagemaker_version})")
    
    # Verificar si es versión 3.x (incompatible)
    if sagemaker_version.startswith('3.'):
        print("¢Å¡Â ¯Â¸Â  Versión 3.x detectada - se reinstalará versión 2.x")
        sagemaker_needs_reinstall = True
    else:
        # Verificar que funcione correctamente
        try:
            import sagemaker
            if not hasattr(sagemaker, 'Session'):
                print("¢Å¡Â ¯Â¸Â  sagemaker.Session no disponible - se reinstalará")
                sagemaker_needs_reinstall = True
        except Exception as e:
            print(f"¢Å¡Â ¯Â¸Â  Error al importar sagemaker: {e}")
            print("   Se reinstalará para solucionar el problema")
            sagemaker_needs_reinstall = True
else:
    sagemaker_needs_reinstall = True

# Instalar/reinstalar sagemaker si es necesario
if sagemaker_needs_reinstall:
    # Desinstalar primero todas las dependencias relacionadas (para evitar conflictos)
    if sagemaker_installed:
        print("\n📝¦ Desinstalando sagemaker y dependencias relacionadas para evitar conflictos...")
        sagemaker_packages = [
            'sagemaker', 'sagemaker-core', 'sagemaker-mlops', 
            'sagemaker-serve', 'sagemaker-train'
        ]
        
        for pkg in sagemaker_packages:
            try:
                subprocess.check_call([
                    sys.executable, "-m", "pip", "uninstall", pkg, "-y"
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            except:
                pass  # Ignorar si no está instalado
        
        print("¢Å“" Paquetes de sagemaker desinstalados")
        
        # Limpiar cache de importación
        modules_to_remove = [m for m in list(sys.modules.keys()) if 'sagemaker' in m]
        for m in modules_to_remove:
            del sys.modules[m]
    
    if not install_package('sagemaker', '<3.0', force_reinstall=False):
        all_installed = False

if all_installed:
    print("\n" + "=" * 70)
    print("¢Å“"¦ VERIFICACIí"N FINAL DE SAGEMAKER")
    print("=" * 70)
    
    # Verificar que sagemaker funcione correctamente
    try:
        # Limpiar cache de importación
        if 'sagemaker' in sys.modules:
            del sys.modules['sagemaker']
        
        import sagemaker
        
        # Verificar atributos
        has_session = hasattr(sagemaker, 'Session')
        has_get_role = hasattr(sagemaker, 'get_execution_role')
        
        if has_session:
            print("¢Å“" sagemaker.Session disponible")
        else:
            print("¢ÂÅ’ sagemaker.Session no disponible")
            raise ImportError("sagemaker.Session no disponible")
        
        if has_get_role:
            print("¢Å“" sagemaker.get_execution_role disponible")
        else:
            print("¢ÂÅ’ sagemaker.get_execution_role no disponible")
            raise ImportError("sagemaker.get_execution_role no disponible")
        
        version = getattr(sagemaker, '__version__', 'desconocida')
        print(f"¢Å“" Versión de sagemaker: {version}")
        
        if version.startswith('3.'):
            print("\n¢Å¡Â ¯Â¸Â  ADVERTENCIA: Aún tienes sagemaker 3.x instalado")
            print("   Ejecuta la celda de diagnóstico (Paso 0.5) para solucionarlo")
        else:
            print("\n¢Å“"¦ sagemaker está correctamente instalado y funcionando")
            
    except Exception as e:
        print(f"¢ÂÅ’ Error verificando sagemaker: {e}")
        print("\n°Å¸"™Â¡ SOLUCIí"N:")
        print("   1. Ejecuta la celda de diagnóstico (Paso 0.5)")
        print("   2. O manualmente:")
        print("      pip uninstall sagemaker sagemaker-core sagemaker-mlops sagemaker-serve sagemaker-train -y")
        print("      pip install 'sagemaker<3.0'")
        print("   3. Reinicia el kernel y vuelve a ejecutar esta celda")
else:
    print("\n¢Å¡Â ¯Â¸Â  Algunas dependencias no se instalaron correctamente")
    print("   Intenta instalarlas manualmente:")
    print("   pip install boto3 'sagemaker<3.0'")


📝¦ VERIFICANDO E INSTALANDO DEPENDENCIAS
Ã¢Å¡Â   NOTA: Se instalará sagemaker 2.x (compatible con este notebook)

Ã¢Å“" boto3 ya está instalado (versión: 1.41.5)
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\tomas\AppData\Local\sagemaker\sagemaker\config.yaml
Ã¢Å“" sagemaker ya está instalado (versión: 2.254.1)

Ã¢Å“"¦ VERIFICACIÃƒ"N FINAL DE SAGEMAKER
Ã¢Å“" sagemaker.Session disponible
Ã¢Å“" sagemaker.get_execution_role disponible
Ã¢Å“" Versión de sagemaker: 2.254.1

Ã¢Å“"¦ sagemaker está correctamente instalado y funcionando


## ¢Å¡â„¢¯Â¸Â Paso 1: Configuración

**IMPORTANTE:** Edita esta celda con tus credenciales AWS y configuración.

### Cómo obtener tus credenciales:

1. Ve a https://console.aws.amazon.com
2. IAM ¢" "™ Usuarios ¢" "™ Tu usuario ¢" "™ Security credentials
3. Create access key
4. Descarga el CSV (solo se muestra una vez)

### Cómo crear el rol IAM:

1. IAM ¢" "™ Roles ¢" "™ Create role
2. Selecciona: "SageMaker"
3. Adjunta políticas:
   - `AmazonSageMakerFullAccess`
   - `AmazonS3FullAccess`
4. Copia el ARN del rol


In [ ]:
# ========================================
# ¢Å¡â„¢¯Â¸Â CONFIGURACIí"N - EDITA ESTOS VALORES
# ========================================

# ========================================
# CREDENCIALES AWS
# ========================================
# OPCIí"N 1: Configurar aquí directamente (recomendado)
AWS_ACCESS_KEY_ID = "-"  # ⬅️ REEMPLAZA con tu Access Key ID
AWS_SECRET_ACCESS_KEY = "-"  # ⬅️ REEMPLAZA con tu Secret Access Key

# OPCIí"N 2: Dejar None y usar variables de entorno o AWS CLI
# Si dejas None, el script buscará:
# - Variables de entorno (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
# - Archivo de credenciales AWS (~/.aws/credentials)
# - AWS CLI configurado

# ========================================
# CONFIGURACIí"N DE SAGEMAKER
# ========================================

# ARN del rol de SageMaker (OBLIGATORIO si ejecutas desde local)
SAGEMAKER_ROLE_ARN = "-"  # ⬅️ REEMPLAZA con el ARN de tu rol

# Bucket S3 (None = usa el bucket por defecto de SageMaker)
S3_BUCKET = None

# Región AWS
AWS_REGION = "us-east-1"  # Cambia si prefieres otra región

# Nombre del endpoint (debe ser único en tu cuenta)
ENDPOINT_NAME = "cnn1d-lstm-ecg-v1-serverless"

# Ruta del modelo (no cambiar a menos que esté en otra ubicación)
from pathlib import Path
MODEL_TAR_PATH = Path("sagemaker_models/cnn1d_lstm_ecg_v1_sagemaker.tar.gz")

# Configuración Serverless
SERVERLESS_MEMORY_MB = 3072  # Memoria en MB (3GB, aumentado para evitar timeouts)
SERVERLESS_MAX_CONCURRENCY = 10  # Máximo de invocaciones simultáneas

print("¢Å“"¦ Configuración cargada")
print(f"   Región: {AWS_REGION}")
print(f"   Endpoint: {ENDPOINT_NAME}")
print(f"   Memoria: {SERVERLESS_MEMORY_MB} MB")


Ã¢Å“"¦ Configuración cargada
   Región: us-east-1
   Endpoint: cnn1d-lstm-ecg-v1-serverless
   Memoria: 3072 MB


## °Å¸"Â§ Diagnóstico Rápido (Si tienes errores)

Si encuentras errores con `sagemaker`, ejecuta esta celda para diagnosticar y solucionar.


In [3]:
# °Å¸"Â§ DIAGNí"STICO Y SOLUCIí"N DE PROBLEMAS CON SAGEMAKER
# Ejecuta esta celda si tienes errores con sagemaker

import sys
import subprocess

print("=" * 70)
print("°Å¸"Â DIAGNí"STICO DE SAGEMAKER")
print("=" * 70)

# Verificar instalación
necesita_reinstalacion = False

try:
    import sagemaker
    print("¢Å“" sagemaker está importado")
    
    # Verificar atributos
    has_session = hasattr(sagemaker, 'Session')
    has_get_role = hasattr(sagemaker, 'get_execution_role')
    
    if has_session:
        print("¢Å“" sagemaker.Session disponible")
    else:
        print("¢ÂÅ’ sagemaker.Session NO disponible")
        print("   Esto indica un problema con la instalación o versión incorrecta")
        necesita_reinstalacion = True
        
    if has_get_role:
        print("¢Å“" sagemaker.get_execution_role disponible")
    else:
        print("¢ÂÅ’ sagemaker.get_execution_role NO disponible")
        necesita_reinstalacion = True
    
    # Verificar versión
    version = getattr(sagemaker, '__version__', 'desconocida')
    print(f"¢Å“" Versión: {version}")
    
    # Verificar si es versión 3.x (incompatible)
    if version.startswith('3.'):
        print("\n¢Å¡Â ¯Â¸Â  PROBLEMA DETECTADO: Tienes sagemaker 3.x instalado")
        print("   Este notebook requiere sagemaker 2.x")
        necesita_reinstalacion = True
    
    if has_session and has_get_role and not version.startswith('3.'):
        print("\n¢Å“"¦ sagemaker está correctamente instalado y configurado")
        print("   No se necesita reinstalación")
    
except ImportError as e:
    print(f"¢ÂÅ’ Error importando sagemaker: {e}")
    print("   sagemaker no está instalado")
    necesita_reinstalacion = True

# Verificar si hay archivos que puedan causar conflicto
import os
from pathlib import Path

current_dir = Path.cwd()
sagemaker_files = list(current_dir.glob("sagemaker.py"))
if sagemaker_files:
    print(f"\n¢Å¡Â ¯Â¸Â  ADVERTENCIA: Se encontró archivo sagemaker.py en {current_dir}")
    print("   Esto puede causar conflictos. Considera renombrarlo.")

# Solución automática (solo si es necesario)
if necesita_reinstalacion:
    print("\n" + "=" * 70)
    print("°Å¸"Â§ INTENTANDO SOLUCIí"N AUTOMíTICA...")
    print("=" * 70)
    print("¢Å¡Â ¯Â¸Â  IMPORTANTE: Se instalará sagemaker 2.x (compatible con este notebook)")
    print()
    
    try:
        print("📝¦ Desinstalando sagemaker y todas sus dependencias...")
        # Desinstalar todos los paquetes relacionados con sagemaker
        sagemaker_packages = [
            'sagemaker', 'sagemaker-core', 'sagemaker-mlops', 
            'sagemaker-serve', 'sagemaker-train'
        ]
        
        for pkg in sagemaker_packages:
            try:
                subprocess.check_call([
                    sys.executable, "-m", "pip", "uninstall", pkg, "-y"
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            except:
                pass  # Ignorar si no está instalado
        
        print("¢Å“" Paquetes de sagemaker desinstalados")
        
        # Limpiar cache de importación
        modules_to_remove = [m for m in list(sys.modules.keys()) if 'sagemaker' in m]
        for m in modules_to_remove:
            del sys.modules[m]
        
        print("📝¦ Instalando sagemaker 2.x (compatible)...")
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", 
            "sagemaker<3.0"
        ])
        print("¢Å“"¦ sagemaker 2.x instalado")
        
        # Verificar nuevamente
        import importlib
        if 'sagemaker' in sys.modules:
            importlib.reload(sys.modules['sagemaker'])
        else:
            import sagemaker
        
        if hasattr(sagemaker, 'Session') and hasattr(sagemaker, 'get_execution_role'):
            version = getattr(sagemaker, '__version__', 'desconocida')
            print(f"\n¢Å“"¦ sagemaker.Session ahora está disponible")
            print(f"¢Å“"¦ Versión instalada: {version}")
            print("\n°Å¸"™Â¡ IMPORTANTE: Reinicia el kernel (Kernel ¢" "™ Restart)")
            print("   Luego vuelve a ejecutar desde el Paso 0")
        else:
            print("¢ÂÅ’ sagemaker.Session aún no está disponible después de la reinstalación")
            print("\n°Å¸"™Â¡ SOLUCIí"N MANUAL:")
            print("   1. Cierra este notebook")
            print("   2. Abre una terminal/PowerShell")
            print("   3. Ejecuta:")
            print("      pip uninstall sagemaker -y")
            print("      pip install 'sagemaker<3.0'")
            print("   4. Vuelve a abrir el notebook y reinicia el kernel")
            
    except Exception as e:
        print(f"¢ÂÅ’ Error durante la reinstalación: {e}")
        print("\n°Å¸"™Â¡ SOLUCIí"N MANUAL:")
        print("   Ejecuta en una terminal:")
        print("   pip uninstall sagemaker -y")
        print("   pip install 'sagemaker<3.0'")
        print("   Luego reinicia el kernel del notebook")
else:
    print("\n¢Å“"¦ sagemaker ya está correctamente instalado (versión 2.x)")
    print("   No se necesita ninguna acción adicional")


Ã°Å¸"Â DIAGNÃƒ"STICO DE SAGEMAKER
Ã¢Å“" sagemaker está importado
Ã¢Å“" sagemaker.Session disponible
Ã¢Å“" sagemaker.get_execution_role disponible
Ã¢Å“" Versión: 2.254.1

Ã¢Å“"¦ sagemaker está correctamente instalado y configurado
   No se necesita reinstalación

Ã¢Å“"¦ sagemaker ya está correctamente instalado (versión 2.x)
   No se necesita ninguna acción adicional


In [4]:
import os
import sys

print("=" * 70)
print("°Å¸"Â VERIFICANDO ARCHIVOS...")
print("=" * 70)

if not MODEL_TAR_PATH.exists():
    print(f"¢ÂÅ’ ERROR: No se encuentra el modelo en: {MODEL_TAR_PATH}")
    print(f"   Ruta absoluta: {MODEL_TAR_PATH.resolve()}")
    print("\n°Å¸"™Â¡ Verifica que el archivo exista en la carpeta sagemaker_models/")
    raise FileNotFoundError(f"Modelo no encontrado: {MODEL_TAR_PATH}")

print(f"¢Å“" Modelo encontrado: {MODEL_TAR_PATH}")
print(f"  Tamaño: {MODEL_TAR_PATH.stat().st_size / (1024*1024):.2f} MB")
print(f"  Ruta absoluta: {MODEL_TAR_PATH.resolve()}")


Ã°Å¸"Â VERIFICANDO ARCHIVOS...
Ã¢Å“" Modelo encontrado: sagemaker_models\cnn1d_lstm_ecg_v1_sagemaker.tar.gz
  Tamaño: 0.60 MB
  Ruta absoluta: S:\Proyecto final\Books\sagemaker_models\cnn1d_lstm_ecg_v1_sagemaker.tar.gz


## °Å¸"Â Paso 3: Configuración y Verificación de Credenciales AWS

Configura las credenciales y verifica que funcionen correctamente.


In [5]:
import boto3

print("=" * 70)
print("°Å¸"Â CONFIGURANDO CREDENCIALES AWS...")
print("=" * 70)

# Configurar credenciales si se proporcionaron directamente
if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
    os.environ['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
    os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY
    if AWS_REGION:
        os.environ['AWS_DEFAULT_REGION'] = AWS_REGION
    print("¢Å“" Credenciales configuradas desde el notebook")
elif os.environ.get('AWS_ACCESS_KEY_ID') and os.environ.get('AWS_SECRET_ACCESS_KEY'):
    print("¢Å“" Credenciales encontradas en variables de entorno")
else:
    print("¢Å¡Â ¯Â¸Â  No se encontraron credenciales en el notebook ni variables de entorno")
    print("   Buscando en archivo de credenciales AWS o AWS CLI...")

# Crear sesión con credenciales
if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
    boto_session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION
    )
else:
    boto_session = boto3.Session(region_name=AWS_REGION)

# Verificar credenciales
print("\n" + "=" * 70)
print("°Å¸"Â VERIFICANDO CREDENCIALES AWS...")
print("=" * 70)

try:
    sts = boto_session.client('sts')
    identity = sts.get_caller_identity()
    print(f"¢Å“"¦ Credenciales AWS válidas")
    print(f"   Account ID: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se pueden verificar las credenciales AWS")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ SOLUCIONES:")
    print("\n   OPCIí"N 1: Configurar en la celda de configuración (líneas 2-3):")
    print("      AWS_ACCESS_KEY_ID = 'tu-access-key'")
    print("      AWS_SECRET_ACCESS_KEY = 'tu-secret-key'")
    print("\n   OPCIí"N 2: Variables de entorno (PowerShell):")
    print("      $env:AWS_ACCESS_KEY_ID='tu-access-key'")
    print("      $env:AWS_SECRET_ACCESS_KEY='tu-secret-key'")
    print("      $env:AWS_DEFAULT_REGION='us-east-1'")
    print("\n   OPCIí"N 3: Instalar AWS CLI y ejecutar 'aws configure'")
    print("      Descarga: https://aws.amazon.com/cli/")
    raise


Ã°Å¸"Â CONFIGURANDO CREDENCIALES AWS...
Ã¢Å“" Credenciales configuradas desde el notebook

Ã°Å¸"Â VERIFICANDO CREDENCIALES AWS...
Ã¢Å“"¦ Credenciales AWS válidas
   Account ID: 235500188303
   User/Role: arn:aws:iam::235500188303:root


## °Å¸"˜Â¤ Paso 4: Configuración del Rol de SageMaker

Verifica o configura el rol IAM que SageMaker usará.


## °Å¸Â§Âª Paso 11: Probar Endpoint Específico

Usa esta celda para probar cualquier endpoint especificando su nombre manualmente.


In [ ]:
# ========================================
# °Å¸Â§Âª PROBAR ENDPOINT ESPECíFICO
# ========================================
# Edita estas variables para probar tu endpoint

# ¢Â¬"¡¯Â¸Â CONFIGURACIí"N - EDITA AQUí
# Puedes usar el nombre del endpoint O la URL completa
ENDPOINT_NAME_OR_URL = "cnn1d-lstm-ecg-v1-serverless"  # ¢Â¬"¦¯Â¸Â Nombre del endpoint O URL completa
AWS_REGION_TO_TEST = "us-east-1"  # ¢Â¬"¦¯Â¸Â Cambia por tu región si es diferente

# Extraer nombre del endpoint si se proporciona una URL
import re
if isinstance(ENDPOINT_NAME_OR_URL, str) and ENDPOINT_NAME_OR_URL.startswith('http'):
    # Es una URL, extraer el nombre del endpoint
    pattern = r'https://runtime\.sagemaker\.([^.]+)\.amazonaws\.com/endpoints/([^/]+)/invocations'
    match = re.match(pattern, ENDPOINT_NAME_OR_URL)
    if match:
        AWS_REGION_TO_TEST = match.group(1)  # Actualizar región desde la URL
        ENDPOINT_NAME_TO_TEST = match.group(2)
        print(f"📝 URL detectada, extrayendo información:")
        print(f"   Nombre del endpoint: {ENDPOINT_NAME_TO_TEST}")
        print(f"   Región: {AWS_REGION_TO_TEST}")
    else:
        ENDPOINT_NAME_TO_TEST = ENDPOINT_NAME_OR_URL
        print(f"¢Å¡Â ¯Â¸Â  URL no reconocida, usando como nombre: {ENDPOINT_NAME_TO_TEST}")
else:
    # Es un nombre de endpoint
    ENDPOINT_NAME_TO_TEST = ENDPOINT_NAME_OR_URL

# Opcional: Cargar ECG desde archivo (deja None para usar datos aleatorios)
ECG_FILE_PATH = "../data/Datos_supervisados/tensors_200hz/X_test.pt"  # Ejemplo: "../data/Datos_supervisados/tensors_200hz/X_test.pt"
ECG_LABELS_PATH = "../data/Datos_supervisados/tensors_200hz/y_test.pt"  # Ruta a las etiquetas (opcional)
ECG_INDEX = None  # índice del ECG a usar (None = aleatorio)

print("=" * 70)
print("°Å¸Â§Âª PROBANDO ENDPOINT ESPECíFICO")
print("=" * 70)
print(f"📝 Endpoint: {ENDPOINT_NAME_TO_TEST}")
print(f"°Å¸Å’Â Región: {AWS_REGION_TO_TEST}")
print()

# ========================================
# Cargar o generar datos de prueba
# ========================================
import json
import numpy as np
import torch
from pathlib import Path
import random

ecg_sample = None
ecg_true_label = None  # Etiqueta real del ECG (0=normal, 1=anómalo)
ecg_index_used = None

# Intentar cargar desde archivo si se especificó
if ECG_FILE_PATH:
    try:
        file_path = Path(ECG_FILE_PATH)
        if not file_path.exists():
            print(f"¢Å¡Â ¯Â¸Â  Archivo no encontrado: {ECG_FILE_PATH}")
            print("   Usando datos aleatorios...")
        else:
            print(f"°Å¸""š Cargando ECG desde: {file_path}")
            
            if file_path.suffix == '.pt':
                # Archivo PyTorch
                data = torch.load(file_path, map_location='cpu')
                if isinstance(data, torch.Tensor):
                    X_test = data.numpy()
                else:
                    X_test = np.array(data)
            elif file_path.suffix == '.npy':
                # Archivo NumPy
                X_test = np.load(file_path, mmap_mode='r')
            else:
                raise ValueError(f"Formato de archivo no soportado: {file_path.suffix}")
            
            num_ecgs = X_test.shape[0]
            idx = ECG_INDEX if ECG_INDEX is not None else random.randint(0, num_ecgs - 1)
            ecg_sample = X_test[idx:idx+1]  # [1, seq_len, n_channels]
            ecg_index_used = idx
            
            # Intentar cargar etiquetas si están disponibles
            if ECG_LABELS_PATH:
                try:
                    labels_path = Path(ECG_LABELS_PATH)
                    if labels_path.exists():
                        if labels_path.suffix == '.pt':
                            y_test = torch.load(labels_path, map_location='cpu')
                            if isinstance(y_test, torch.Tensor):
                                y_test = y_test.numpy()
                            else:
                                y_test = np.array(y_test)
                        elif labels_path.suffix == '.npy':
                            y_test = np.load(labels_path, mmap_mode='r')
                        else:
                            y_test = None
                        
                        if y_test is not None and len(y_test) > idx:
                            ecg_true_label = int(y_test[idx]) if isinstance(y_test[idx], (int, float)) else int(y_test[idx].item())
                    else:
                        # Intentar encontrar y_test.pt en el mismo directorio
                        labels_path = file_path.parent / "y_test.pt"
                        if labels_path.exists():
                            y_test = torch.load(labels_path, map_location='cpu')
                            if isinstance(y_test, torch.Tensor):
                                y_test = y_test.numpy()
                            else:
                                y_test = np.array(y_test)
                            if len(y_test) > idx:
                                ecg_true_label = int(y_test[idx]) if isinstance(y_test[idx], (int, float)) else int(y_test[idx].item())
                except Exception as e:
                    print(f"   ¢Å¡Â ¯Â¸Â  No se pudieron cargar las etiquetas: {e}")
            
            print(f"¢Å“"¦ ECG cargado exitosamente")
            print(f"   Total de ECGs: {num_ecgs}")
            print(f"   ECG seleccionado: índice {idx}")
            print(f"   Forma: {ecg_sample.shape}")
            print(f"   Rango: [{ecg_sample.min():.4f}, {ecg_sample.max():.4f}]")
            if ecg_true_label is not None:
                label_text = "ANí"MALO" if ecg_true_label == 1 else "NORMAL"
                print(f"   °Å¸ÂÂ·¯Â¸Â  Etiqueta real: {ecg_true_label} ({label_text})")
            
    except Exception as e:
        print(f"¢Å¡Â ¯Â¸Â  Error cargando archivo: {e}")
        print("   Usando datos aleatorios...")

# Si no se cargó, usar datos aleatorios
if ecg_sample is None:
    print("°Å¸"Å  Generando datos aleatorios de prueba...")
    ecg_sample = np.random.randn(1, 2000, 3).astype(np.float32)
    print(f"   Forma: {ecg_sample.shape}")
    print(f"   Rango: [{ecg_sample.min():.4f}, {ecg_sample.max():.4f}]")

# Verificar y ajustar forma si es necesario
if ecg_sample.shape[1] != 2000 or ecg_sample.shape[2] != 3:
    print(f"\n¢Å¡Â ¯Â¸Â  Ajustando forma de {ecg_sample.shape} a [1, 2000, 3]...")
    if ecg_sample.shape[1] > 2000:
        ecg_sample = ecg_sample[:, :2000, :]
    elif ecg_sample.shape[1] < 2000:
        padding = np.zeros((1, 2000 - ecg_sample.shape[1], 3), dtype=ecg_sample.dtype)
        ecg_sample = np.concatenate([ecg_sample, padding], axis=1)
    if ecg_sample.shape[2] != 3:
        if ecg_sample.shape[2] > 3:
            ecg_sample = ecg_sample[:, :, :3]
        else:
            padding = np.zeros((1, ecg_sample.shape[1], 3 - ecg_sample.shape[2]), dtype=ecg_sample.dtype)
            ecg_sample = np.concatenate([ecg_sample, padding], axis=2)
    print(f"   Forma ajustada: {ecg_sample.shape}")

# ========================================
# Preparar y enviar datos al endpoint
# ========================================
print(f"\n📝¤ Enviando datos al endpoint...")
print(f"   Forma final: {ecg_sample.shape}")

try:
    import boto3
    import os
    
    # Crear cliente de SageMaker Runtime con credenciales
    # Intentar usar las credenciales del notebook primero
    try:
        if 'AWS_ACCESS_KEY_ID' in globals() and 'AWS_SECRET_ACCESS_KEY' in globals():
            if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
                runtime = boto3.client(
                    'sagemaker-runtime',
                    region_name=AWS_REGION_TO_TEST,
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
                )
            else:
                runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION_TO_TEST)
        else:
            runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION_TO_TEST)
    except NameError:
        # Si las variables no están definidas, usar las de entorno o por defecto
        if os.environ.get('AWS_ACCESS_KEY_ID') and os.environ.get('AWS_SECRET_ACCESS_KEY'):
            runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION_TO_TEST)
        else:
            runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION_TO_TEST)
    
    # Preparar datos en formato JSON
    test_data = {
        "signals": ecg_sample.tolist()
    }
    
    # Serializar JSON
    json_body = json.dumps(test_data, ensure_ascii=False)
    print(f"   Tamaño del JSON: {len(json_body)} bytes")
    
    # Invocar endpoint
    print(f"   Invocando endpoint (esto puede tardar unos segundos)...")
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME_TO_TEST,
        ContentType='application/json',
        Body=json_body.encode('utf-8')
    )
    
    # Leer respuesta
    result = json.loads(response['Body'].read().decode('utf-8'))
    
    print(f"\n¢Å“"¦ Endpoint respondió correctamente!")
    print(f"\n°Å¸"Å  RESPUESTA DEL ENDPOINT:")
    print(json.dumps(result, indent=2))
    
    # Extraer valor de predicción
    pred_value = None
    if isinstance(result, dict):
        if 'prediction' in result:
            pred = result['prediction']
            if isinstance(pred, (int, float)):
                pred_value = float(pred)
            elif isinstance(pred, list) and len(pred) > 0:
                pred_value = float(pred[0]) if isinstance(pred[0], (int, float)) else float(pred[0][0]) if isinstance(pred[0], list) else float(pred)
        elif 'predictions' in result:
            preds = result['predictions']
            if isinstance(preds, list) and len(preds) > 0:
                pred_value = float(preds[0]) if isinstance(preds[0], (int, float)) else float(preds[0][0]) if isinstance(preds[0], list) else float(preds)
        elif 'probability' in result:
            pred_value = float(result['probability'])
    elif isinstance(result, (list, tuple)):
        if len(result) > 0:
            pred_value = float(result[0]) if isinstance(result[0], (int, float)) else float(result[0][0]) if isinstance(result[0], list) else float(result)
    
    # Interpretar la respuesta
    print(f"\n°Å¸"Â INTERPRETACIí"N:")
    if pred_value is not None:
        # Determinar predicción binaria (threshold = 0.5)
        pred_binary = 1 if pred_value > 0.5 else 0
        pred_label = "ANí"MALO" if pred_binary == 1 else "NORMAL"
        
        print(f"   °Å¸"Å  Probabilidad de anomalía: {pred_value:.4f}")
        print(f"   °Å¸Å½Â¯ Predicción del modelo: {pred_binary} ({pred_label})")
        
        # Si tenemos la etiqueta real, comparar
        if ecg_true_label is not None:
            true_label_text = "ANí"MALO" if ecg_true_label == 1 else "NORMAL"
            print(f"\n   📋 COMPARACIí"N CON VERDAD DEL TERRENO:")
            print(f"      °Å¸ÂÂ·¯Â¸Â  Etiqueta real: {ecg_true_label} ({true_label_text})")
            print(f"      °Å¸Â¤"“ Predicción: {pred_binary} ({pred_label})")
            
            # Verificar si la predicción es correcta
            is_correct = (pred_binary == ecg_true_label)
            if is_correct:
                print(f"      ¢Å“"¦ PREDICCIí"N CORRECTA!")
            else:
                print(f"      ¢ÂÅ’ PREDICCIí"N INCORRECTA")
                if ecg_true_label == 1 and pred_binary == 0:
                    print(f"         Tipo de error: Falso Negativo (no detectó anomalía)")
                elif ecg_true_label == 0 and pred_binary == 1:
                    print(f"         Tipo de error: Falso Positivo (detectó anomalía cuando no la hay)")
        else:
            print(f"\n   °Å¸"™Â¡ No se pudo cargar la etiqueta real para comparar")
            print(f"      (Asegúrate de que y_test.pt esté en el mismo directorio que X_test.pt)")
    else:
        print(f"   ¢Å¡Â ¯Â¸Â  No se pudo extraer el valor de predicción de la respuesta")
        print(f"   Respuesta recibida: {result}")
    
    # Información adicional de la respuesta
    print(f"\n°Å¸"Ë† METADATOS DE LA RESPUESTA:")
    print(f"   Status Code: {response['ResponseMetadata']['HTTPStatusCode']}")
    print(f"   Content Type: {response.get('ContentType', 'N/A')}")
    print(f"   Tamaño de respuesta: {len(json.dumps(result))} bytes")
    
except Exception as e:
    print(f"\n¢ÂÅ’ ERROR al invocar el endpoint")
    print(f"   Detalle: {e}")
    print(f"\n°Å¸"™Â¡ SOLUCIONES:")
    print(f"   1. Verifica que el endpoint '{ENDPOINT_NAME_TO_TEST}' exista")
    print(f"   2. Verifica que esté en estado 'InService'")
    print(f"   3. Verifica la región: {AWS_REGION_TO_TEST}")
    print(f"   4. Si proporcionaste una URL, asegúrate de que el formato sea correcto")
    print(f"   5. Revisa los logs en CloudWatch:")
    print(f"      /aws/sagemaker/Endpoints/{ENDPOINT_NAME_TO_TEST}")
    print(f"\n°Å¸""” Enlaces útiles:")
    print(f"   - Consola SageMaker: https://console.aws.amazon.com/sagemaker/home?region={AWS_REGION_TO_TEST}#/endpoints")
    print(f"   - CloudWatch Logs: https://console.aws.amazon.com/cloudwatch/home?region={AWS_REGION_TO_TEST}#logsV2:log-groups/log-group/$252Faws$252Fsagemaker$252FEndpoints$252F{ENDPOINT_NAME_TO_TEST.replace('-', '%2D')}")
    
    # Intentar verificar el estado del endpoint
    try:
        sagemaker_client = boto3.client('sagemaker', region_name=AWS_REGION_TO_TEST)
        endpoint_info = sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME_TO_TEST)
        status = endpoint_info['EndpointStatus']
        print(f"\n°Å¸"Å  Estado del endpoint: {status}")
        if status != 'InService':
            print(f"   ¢Å¡Â ¯Â¸Â  El endpoint no está en estado 'InService'")
            print(f"   Espera a que termine de crearse o actualizarse")
    except Exception as verify_error:
        print(f"\n¢Å¡Â ¯Â¸Â  No se pudo verificar el estado del endpoint: {verify_error}")

print(f"\n" + "=" * 70)
print(f"¢Å“"¦ Prueba completada")
print("=" * 70)


Ã°Å¸Â§Âª PROBANDO ENDPOINT ESPECíFICO
📝 Endpoint: cnn1d-lstm-ecg-v1-serverless
Ã°Å¸Å’Â Región: us-east-1

Ã°Å¸""š Cargando ECG desde: ..\data\Datos_supervisados\tensors_200hz\X_test.pt
Ã¢Å“"¦ ECG cargado exitosamente
   Total de ECGs: 58001
   ECG seleccionado: índice 19096
   Forma: (1, 2000, 3)
   Rango: [0.0014, 1.0001]
   Ã°Å¸ÂÂ·  Etiqueta real: 1 (ANÃƒ"MALO)

📝¤ Enviando datos al endpoint...
   Forma final: (1, 2000, 3)
   Tamaño del JSON: 123855 bytes
   Invocando endpoint (esto puede tardar unos segundos)...

Ã¢Å“"¦ Endpoint respondió correctamente!

Ã°Å¸"Å  RESPUESTA DEL ENDPOINT:
{
  "prediction": 0.9999439716339111,
  "probability": 0.9999439716339111
}

Ã°Å¸"Â INTERPRETACIÃƒ"N:
   Ã°Å¸"Å  Probabilidad de anomalía: 0.9999
   Ã°Å¸Å½Â¯ Predicción del modelo: 1 (ANÃƒ"MALO)

   📋 COMPARACIÃƒ"N CON VERDAD DEL TERRENO:
      Ã°Å¸ÂÂ·  Etiqueta real: 1 (ANÃƒ"MALO)
      Ã°Å¸Â¤"“ Predicción: 1 (ANÃƒ"MALO)
      Ã¢Å“"¦ PREDICCIÃƒ"N CORRECTA!

Ã°Å¸"Ë† METADATOS DE LA RESPUESTA:

## ¢Å“"¦ Verificar y Probar Endpoint desde URL

Si tienes la URL completa del endpoint, usa esta celda para verificar y probarlo.


In [15]:
# ========================================
# ¢Å“"¦ VERIFICAR Y PROBAR ENDPOINT DESDE URL
# ========================================

# ¢Â¬"¡¯Â¸Â PEGA TU URL DEL ENDPOINT AQUí
ENDPOINT_URL = "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/cnn1d-lstm-ecg-v1-serverless/invocations"

print("=" * 70)
print("¢Å“"¦ VERIFICANDO ENDPOINT DESDE URL")
print("=" * 70)

# Extraer información de la URL
import re
from urllib.parse import urlparse

try:
    parsed_url = urlparse(ENDPOINT_URL)
    
    # Extraer región y nombre del endpoint de la URL
    # Formato: https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{endpoint-name}/invocations
    pattern = r'https://runtime\.sagemaker\.([^.]+)\.amazonaws\.com/endpoints/([^/]+)/invocations'
    match = re.match(pattern, ENDPOINT_URL)
    
    if match:
        region = match.group(1)
        endpoint_name = match.group(2)
        
        print(f"📝 URL analizada:")
        print(f"   Región: {region}")
        print(f"   Nombre del endpoint: {endpoint_name}")
        print(f"   URL completa: {ENDPOINT_URL}")
        print()
        
        # Verificar el endpoint
        import boto3
        import os
        
        print("°Å¸"Â Verificando estado del endpoint...")
        
        # Configurar credenciales si están disponibles en el notebook
        # Intentar usar las credenciales de la celda de configuración
        try:
            # Verificar si las variables están definidas en el namespace global
            if 'AWS_ACCESS_KEY_ID' in globals() and 'AWS_SECRET_ACCESS_KEY' in globals():
                if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
                    print("   Usando credenciales del notebook...")
                    sagemaker_client = boto3.client(
                        'sagemaker',
                        region_name=region,
                        aws_access_key_id=AWS_ACCESS_KEY_ID,
                        aws_secret_access_key=AWS_SECRET_ACCESS_KEY
                    )
                else:
                    print("   Usando credenciales por defecto de boto3...")
                    sagemaker_client = boto3.client('sagemaker', region_name=region)
            else:
                # Si no están en globals, intentar desde variables de entorno
                if os.environ.get('AWS_ACCESS_KEY_ID') and os.environ.get('AWS_SECRET_ACCESS_KEY'):
                    print("   Usando credenciales de variables de entorno...")
                    sagemaker_client = boto3.client('sagemaker', region_name=region)
                else:
                    print("   Usando credenciales por defecto de boto3...")
                    sagemaker_client = boto3.client('sagemaker', region_name=region)
        except NameError:
            # Si las variables no están definidas, usar las de entorno o por defecto
            if os.environ.get('AWS_ACCESS_KEY_ID') and os.environ.get('AWS_SECRET_ACCESS_KEY'):
                print("   Usando credenciales de variables de entorno...")
                sagemaker_client = boto3.client('sagemaker', region_name=region)
            else:
                print("   Usando credenciales por defecto de boto3...")
                sagemaker_client = boto3.client('sagemaker', region_name=region)
        
        try:
            endpoint_info = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
            status = endpoint_info['EndpointStatus']
            creation_time = endpoint_info['CreationTime']
            endpoint_arn = endpoint_info['EndpointArn']
            
            print(f"¢Å“"¦ Endpoint encontrado!")
            print(f"   Estado: {status}")
            print(f"   Creado: {creation_time}")
            print(f"   ARN: {endpoint_arn}")
            print()
            
            if status == 'InService':
                print("¢Å“"¦ El endpoint está en servicio y listo para recibir peticiones")
                
                # Probar el endpoint
                print("\n" + "=" * 70)
                print("°Å¸Â§Âª PROBANDO EL ENDPOINT")
                print("=" * 70)
                
                import json
                import numpy as np
                
                # Generar datos de prueba
                print("°Å¸"Å  Generando datos de prueba...")
                ecg_sample = np.random.randn(1, 2000, 3).astype(np.float32)
                print(f"   Forma: {ecg_sample.shape}")
                
                # Preparar datos
                test_data = {
                    "signals": ecg_sample.tolist()
                }
                
                # Invocar usando la URL completa
                print(f"\n📝¤ Invocando endpoint usando la URL...")
                print(f"   URL: {ENDPOINT_URL}")
                
                import requests
                
                json_body = json.dumps(test_data, ensure_ascii=False)
                
                # Obtener credenciales AWS para la petición
                # Intentar usar las credenciales del notebook primero
                try:
                    if 'AWS_ACCESS_KEY_ID' in globals() and 'AWS_SECRET_ACCESS_KEY' in globals():
                        if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
                            session = boto3.Session(
                                aws_access_key_id=AWS_ACCESS_KEY_ID,
                                aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                region_name=region
                            )
                        else:
                            session = boto3.Session(region_name=region)
                    else:
                        session = boto3.Session(region_name=region)
                except NameError:
                    session = boto3.Session(region_name=region)
                
                credentials = session.get_credentials()
                
                if not credentials:
                    raise ValueError("No se encontraron credenciales AWS. Configura AWS_ACCESS_KEY_ID y AWS_SECRET_ACCESS_KEY en la celda de configuración.")
                
                # Crear firma de AWS para la petición
                from botocore.auth import SigV4Auth
                from botocore.awsrequest import AWSRequest
                
                request = AWSRequest(method='POST', url=ENDPOINT_URL, data=json_body.encode('utf-8'))
                request.headers['Content-Type'] = 'application/json'
                
                SigV4Auth(credentials, 'sagemaker', region).add_auth(request)
                
                # Hacer la petición
                response = requests.post(
                    ENDPOINT_URL,
                    data=json_body.encode('utf-8'),
                    headers=dict(request.headers)
                )
                
                if response.status_code == 200:
                    result = response.json()
                    print(f"\n¢Å“"¦ Endpoint respondió correctamente!")
                    print(f"\n°Å¸"Å  RESPUESTA:")
                    print(json.dumps(result, indent=2))
                    
                    # Interpretar respuesta
                    print(f"\n°Å¸"Â INTERPRETACIí"N:")
                    if isinstance(result, dict):
                        if 'prediction' in result:
                            pred = result['prediction']
                            if isinstance(pred, list) and len(pred) > 0:
                                pred_value = pred[0] if isinstance(pred[0], (int, float)) else pred[0][0] if isinstance(pred[0], list) else pred
                                if isinstance(pred_value, (int, float)):
                                    print(f"   Probabilidad: {pred_value:.4f}")
                                    if pred_value > 0.5:
                                        print(f"   °Å¸Å¡Â¨ Resultado: ECG ANí"MALO (probabilidad: {pred_value:.4f})")
                                    else:
                                        print(f"   ¢Å“"¦ Resultado: ECG NORMAL (probabilidad: {1-pred_value:.4f})")
                        elif 'predictions' in result:
                            preds = result['predictions']
                            if isinstance(preds, list) and len(preds) > 0:
                                pred_value = preds[0] if isinstance(preds[0], (int, float)) else preds[0][0] if isinstance(preds[0], list) else preds
                                if isinstance(pred_value, (int, float)):
                                    print(f"   Probabilidad: {pred_value:.4f}")
                                    if pred_value > 0.5:
                                        print(f"   °Å¸Å¡Â¨ Resultado: ECG ANí"MALO (probabilidad: {pred_value:.4f})")
                                    else:
                                        print(f"   ¢Å“"¦ Resultado: ECG NORMAL (probabilidad: {1-pred_value:.4f})")
                    elif isinstance(result, (list, tuple)):
                        if len(result) > 0:
                            pred_value = result[0] if isinstance(result[0], (int, float)) else result[0][0] if isinstance(result[0], list) else result
                            if isinstance(pred_value, (int, float)):
                                print(f"   Probabilidad: {pred_value:.4f}")
                                if pred_value > 0.5:
                                    print(f"   °Å¸Å¡Â¨ Resultado: ECG ANí"MALO (probabilidad: {pred_value:.4f})")
                                else:
                                    print(f"   ¢Å“"¦ Resultado: ECG NORMAL (probabilidad: {1-pred_value:.4f})")
                    
                    print(f"\n°Å¸"Ë† METADATOS:")
                    print(f"   Status Code: {response.status_code}")
                    print(f"   Tamaño de respuesta: {len(json.dumps(result))} bytes")
                    
                else:
                    print(f"\n¢ÂÅ’ Error en la respuesta del endpoint")
                    print(f"   Status Code: {response.status_code}")
                    print(f"   Respuesta: {response.text}")
                    
            else:
                print(f"¢Å¡Â ¯Â¸Â  El endpoint NO está en servicio")
                print(f"   Estado actual: {status}")
                print(f"   Espera a que el estado cambie a 'InService'")
                
        except Exception as e:
            # Manejar diferentes tipos de errores de AWS
            from botocore.exceptions import ClientError
            
            if isinstance(e, ClientError):
                error_code = e.response.get('Error', {}).get('Code', 'Unknown')
                error_message = e.response.get('Error', {}).get('Message', str(e))
                
                if error_code == 'ValidationException' or 'not found' in error_message.lower() or error_code == 'ResourceNotFound':
                    print(f"¢ÂÅ’ Endpoint no encontrado: {endpoint_name}")
                    print(f"   Verifica que el nombre del endpoint sea correcto")
                    print(f"   Código de error: {error_code}")
                    print(f"   Mensaje: {error_message}")
                else:
                    print(f"¢ÂÅ’ Error verificando el endpoint")
                    print(f"   Código de error: {error_code}")
                    print(f"   Mensaje: {error_message}")
            else:
                print(f"¢ÂÅ’ Error verificando el endpoint: {e}")
                print(f"   Tipo de error: {type(e).__name__}")
            
    else:
        print(f"¢ÂÅ’ URL no válida o formato no reconocido")
        print(f"   Formato esperado: https://runtime.sagemaker.{{region}}.amazonaws.com/endpoints/{{endpoint-name}}/invocations")
        print(f"   URL recibida: {ENDPOINT_URL}")
        
except Exception as e:
    print(f"¢ÂÅ’ Error procesando la URL: {e}")
    
    # Si es un error de credenciales, dar instrucciones claras
    if 'credentials' in str(e).lower() or 'NoCredentialsError' in str(type(e).__name__):
        print(f"\n°Å¸"™Â¡ PROBLEMA: No se encontraron credenciales AWS")
        print(f"\n📝 SOLUCIí"N:")
        print(f"   1. Ejecuta la celda de CONFIGURACIí"N (Paso 1) donde defines:")
        print(f"      AWS_ACCESS_KEY_ID = 'tu-access-key'")
        print(f"      AWS_SECRET_ACCESS_KEY = 'tu-secret-key'")
        print(f"\n   2. O configura variables de entorno:")
        print(f"      $env:AWS_ACCESS_KEY_ID='tu-access-key'")
        print(f"      $env:AWS_SECRET_ACCESS_KEY='tu-secret-key'")
        print(f"\n   3. O instala AWS CLI y ejecuta 'aws configure'")
        print(f"\n¢Å¡Â ¯Â¸Â  IMPORTANTE: NO necesitas hacer el endpoint público.")
        print(f"   Solo necesitas credenciales AWS válidas para acceder a tu propio endpoint.")
    else:
        import traceback
        traceback.print_exc()

print("\n" + "=" * 70)
print("¢Å“"¦ Verificación completada")
print("=" * 70)

# También actualizar la celda anterior con estos valores
print("\n°Å¸"™Â¡ TIP: Puedes usar estos valores en la celda 'Paso 11' para probar el endpoint:")
print(f"   ENDPOINT_NAME_TO_TEST = \"{endpoint_name if 'endpoint_name' in locals() else 'cnn1d-lstm-ecg-v1-serverless'}\"")
print(f"   AWS_REGION_TO_TEST = \"{region if 'region' in locals() else 'us-east-1'}\"")


Ã¢Å“"¦ VERIFICANDO ENDPOINT DESDE URL
📝 URL analizada:
   Región: us-east-1
   Nombre del endpoint: cnn1d-lstm-ecg-v1-serverless
   URL completa: https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/cnn1d-lstm-ecg-v1-serverless/invocations

Ã°Å¸"Â Verificando estado del endpoint...
   Usando credenciales del notebook...
Ã¢Å“"¦ Endpoint encontrado!
   Estado: InService
   Creado: 2025-11-28 23:42:06.046000-03:00
   ARN: arn:aws:sagemaker:us-east-1:235500188303:endpoint/cnn1d-lstm-ecg-v1-serverless

Ã¢Å“"¦ El endpoint está en servicio y listo para recibir peticiones

Ã°Å¸Â§Âª PROBANDO EL ENDPOINT
Ã°Å¸"Å  Generando datos de prueba...
   Forma: (1, 2000, 3)

📝¤ Invocando endpoint usando la URL...
   URL: https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/cnn1d-lstm-ecg-v1-serverless/invocations

Ã¢Å“"¦ Endpoint respondió correctamente!

Ã°Å¸"Å  RESPUESTA:
{
  "prediction": 0.9394499659538269,
  "probability": 0.9394499659538269
}

Ã°Å¸"Â INTERPRETACIÃƒ"N:

Ã°Å¸"Ë† META

In [6]:
import sagemaker

print("=" * 70)
print("°Å¸"˜Â¤ CONFIGURANDO ROL DE SAGEMAKER...")
print("=" * 70)

try:
    if SAGEMAKER_ROLE_ARN is None:
        # Intentar obtener el rol automáticamente (si estás en SageMaker)
        role = sagemaker.get_execution_role()
        print(f"¢Å“"¦ Rol obtenido automáticamente: {role}")
    else:
        role = SAGEMAKER_ROLE_ARN
        print(f"¢Å“"¦ Usando rol especificado: {role}")
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se puede obtener el rol de SageMaker")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ SOLUCIí"N:")
    print("   1. Si estás en tu máquina local, especifica SAGEMAKER_ROLE_ARN en la celda de configuración")
    print("   2. El rol debe tener permisos:")
    print("      - AmazonSageMakerFullAccess")
    print("      - AmazonS3FullAccess (o permisos en tu bucket)")
    print("   3. Crea el rol en IAM ¢" "™ Roles ¢" "™ Create role ¢" "™ SageMaker")
    print("   4. Copia el ARN del rol y pégalo en SAGEMAKER_ROLE_ARN")
    raise


Ã°Å¸"˜Â¤ CONFIGURANDO ROL DE SAGEMAKER...
Ã¢Å“"¦ Usando rol especificado: None  # Ã¢Â¬"¦ REEMPLAZA con el ARN de tu rol (ejemplo: "arn:aws:iam::123456789012:role/SageMakerExecutionRole")


## 📝¦ Paso 5: Configuración de Sesión de SageMaker

Configura la sesión de SageMaker y el bucket S3.


In [7]:
print("=" * 70)
print("📝¦ CONFIGURANDO SESIí"N DE SAGEMAKER...")
print("=" * 70)

# Verificar que sagemaker esté correctamente importado
try:
    import sagemaker
    from sagemaker import Session
    
    # Verificar que Session esté disponible
    if not hasattr(sagemaker, 'Session'):
        print("¢ÂÅ’ ERROR: sagemaker.Session no está disponible")
        print("\n°Å¸"™Â¡ SOLUCIí"N:")
        print("   1. Reinstala sagemaker:")
        print("      pip uninstall sagemaker -y")
        print("      pip install sagemaker")
        print("\n   2. O ejecuta esta celda para reinstalar automáticamente:")
        print("      !pip install --upgrade --force-reinstall sagemaker")
        raise AttributeError("sagemaker.Session no disponible")
    
    sagemaker_session = sagemaker.Session(
        default_bucket=S3_BUCKET,
        boto_session=boto_session
    )
    bucket = sagemaker_session.default_bucket()
    print(f"¢Å“"¦ Bucket S3 configurado: {bucket}")
    print(f"   Región: {AWS_REGION}")
    
except AttributeError as e:
    print(f"¢ÂÅ’ ERROR: {e}")
    print("\n°Å¸"™Â¡ Reinstalando sagemaker automáticamente...")
    import subprocess
    import sys
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "sagemaker"])
        print("¢Å“"¦ sagemaker reinstalado. Por favor, ejecuta esta celda nuevamente.")
        print("   O reinicia el kernel y vuelve a ejecutar desde el inicio.")
    except Exception as install_error:
        print(f"¢ÂÅ’ Error al reinstalar: {install_error}")
        print("\n°Å¸"™Â¡ Reinstala manualmente:")
        print("   pip uninstall sagemaker -y")
        print("   pip install sagemaker")
    raise
    
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se puede configurar la sesión de SageMaker")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ Verifica:")
    print("   - Que sagemaker esté instalado: pip list | grep sagemaker")
    print("   - Que no haya conflictos de nombres (archivo sagemaker.py en el directorio)")
    print("   - Reinstala sagemaker: pip install --upgrade --force-reinstall sagemaker")
    raise


📝¦ CONFIGURANDO SESIÃƒ"N DE SAGEMAKER...
Ã¢Å“"¦ Bucket S3 configurado: sagemaker-us-east-1-235500188303
   Región: us-east-1


## °Å¸"Â§ Recrear Modelo .tar.gz con Código Actualizado

Si actualizaste el código de inferencia (`inference.py`), ejecuta esta celda para recrear el archivo `.tar.gz` con el código actualizado.


In [45]:
# Recrear el archivo .tar.gz con el código actualizado
import tarfile
import shutil
from pathlib import Path

print("=" * 70)
print("°Å¸"Â§ RECREANDO MODELO .TAR.GZ CON Cí"DIGO ACTUALIZADO...")
print("=" * 70)

# Rutas
model_dir = Path("sagemaker_models/sagemaker_cnn1d_lstm_ecg_v1")
output_tar = Path("sagemaker_models/cnn1d_lstm_ecg_v1_sagemaker.tar.gz")

# Verificar que el directorio existe
if not model_dir.exists():
    print(f"¢ÂÅ’ ERROR: No se encontró el directorio {model_dir}")
    raise FileNotFoundError(f"Directorio no encontrado: {model_dir}")

print(f"°Å¸""š Directorio del modelo: {model_dir}")
print(f"📝¦ Archivo de salida: {output_tar}")

# Verificar archivos necesarios
required_files = [
    model_dir / "model.pth",
    model_dir / "config.json",
    model_dir / "code" / "inference.py",
    model_dir / "code" / "requirements.txt"
]

print("\n°Å¸"Â Verificando archivos necesarios...")
for file in required_files:
    if file.exists():
        print(f"   ¢Å“"¦ {file.name}")
    else:
        print(f"   ¢ÂÅ’ {file.name} - NO ENCONTRADO")
        raise FileNotFoundError(f"Archivo requerido no encontrado: {file}")

# Eliminar archivo anterior si existe
if output_tar.exists():
    print(f"\n°Å¸"”"˜¯Â¸Â  Eliminando archivo anterior: {output_tar}")
    output_tar.unlink()

# Crear el archivo .tar.gz
print(f"\n📝¦ Creando archivo .tar.gz...")
print("   Esto puede tardar unos segundos...")

with tarfile.open(output_tar, "w:gz") as tar:
    # Agregar model.pth
    print("   Agregando model.pth...")
    tar.add(model_dir / "model.pth", arcname="model.pth")
    
    # Agregar config.json
    print("   Agregando config.json...")
    tar.add(model_dir / "config.json", arcname="config.json")
    
    # Agregar code/inference.py
    print("   Agregando code/inference.py...")
    tar.add(model_dir / "code" / "inference.py", arcname="code/inference.py")
    
    # Agregar code/requirements.txt
    print("   Agregando code/requirements.txt...")
    tar.add(model_dir / "code" / "requirements.txt", arcname="code/requirements.txt")

print(f"\n¢Å“"¦ Archivo .tar.gz creado exitosamente: {output_tar}")

# Verificar estructura
print("\n°Å¸"Â Verificando estructura del archivo...")
with tarfile.open(output_tar, "r:gz") as tar:
    members = tar.getnames()
    print(f"   Archivos en tar.gz: {len(members)}")
    for member in sorted(members):
        print(f"   - {member}")
    
    has_code_inference = any('code/inference.py' in m or m == 'code/inference.py' for m in members)
    has_model = any('model.pth' in m or m == 'model.pth' for m in members)
    
    if has_code_inference and has_model:
        print("\n¢Å“"¦ Estructura correcta - listo para subir a S3")
    else:
        print("\n¢ÂÅ’ ERROR: Estructura incorrecta")
        if not has_code_inference:
            print("   Falta: code/inference.py")
        if not has_model:
            print("   Falta: model.pth")

print(f"\n°Å¸"™Â¡ Ahora puedes ejecutar la celda 'Sobrescribir Modelo en S3' para subir este archivo actualizado")


Ã°Å¸"Â§ RECREANDO MODELO .TAR.GZ CON CÃƒ"DIGO ACTUALIZADO...
Ã°Å¸""š Directorio del modelo: sagemaker_models\sagemaker_cnn1d_lstm_ecg_v1
📝¦ Archivo de salida: sagemaker_models\cnn1d_lstm_ecg_v1_sagemaker.tar.gz

Ã°Å¸"Â Verificando archivos necesarios...
   Ã¢Å“"¦ model.pth
   Ã¢Å“"¦ config.json
   Ã¢Å“"¦ inference.py
   Ã¢Å“"¦ requirements.txt

Ã°Å¸"”"˜  Eliminando archivo anterior: sagemaker_models\cnn1d_lstm_ecg_v1_sagemaker.tar.gz

📝¦ Creando archivo .tar.gz...
   Esto puede tardar unos segundos...
   Agregando model.pth...
   Agregando config.json...
   Agregando code/inference.py...
   Agregando code/requirements.txt...

Ã¢Å“"¦ Archivo .tar.gz creado exitosamente: sagemaker_models\cnn1d_lstm_ecg_v1_sagemaker.tar.gz

Ã°Å¸"Â Verificando estructura del archivo...
   Archivos en tar.gz: 4
   - code/inference.py
   - code/requirements.txt
   - config.json
   - model.pth

Ã¢Å“"¦ Estructura correcta - listo para subir a S3

Ã°Å¸"™Â¡ Ahora puedes ejecutar la celda 'Sobrescribir Mode

## °Å¸""ž Sobrescribir Modelo en S3 (Si es necesario)

Si el modelo anterior tenía estructura incorrecta, ejecuta esta celda para sobrescribirlo con la versión corregida.


In [46]:
# Sobrescribir modelo en S3 con la versión corregida
print("=" * 70)
print("°Å¸""ž SOBRESCRIBIENDO MODELO EN S3...")
print("=" * 70)

# Verificar estructura del tar.gz
print("°Å¸"Â Verificando estructura del modelo local...")
import tarfile
try:
    with tarfile.open(MODEL_TAR_PATH, 'r:gz') as tar:
        members = tar.getnames()
        has_code_inference = any('code/inference.py' in m or m == 'code/inference.py' for m in members)
        has_model = any('model.pth' in m or m == 'model.pth' for m in members)
        has_wrong_structure = any(m.startswith('sagemaker_') and '/' not in m.replace('sagemaker_', '') for m in members)
        
        print(f"   Archivos en tar.gz: {len(members)}")
        print(f"   Tiene code/inference.py: {has_code_inference}")
        print(f"   Tiene model.pth: {has_model}")
        print(f"   Estructura incorrecta: {has_wrong_structure}")
        
        if has_code_inference and has_model and not has_wrong_structure:
            print("¢Å“"¦ Estructura correcta - listo para subir")
        else:
            if has_wrong_structure:
                print("¢ÂÅ’ ERROR: Estructura incorrecta detectada")
                print("   El tar.gz tiene el nombre de la carpeta padre")
                raise ValueError("Estructura incorrecta")
except Exception as e:
    print(f"¢ÂÅ’ Error verificando estructura: {e}")
    raise

# Subir modelo (sobrescribe si existe)
s3_key = f"models/cnn1d_lstm_ecg_v1/{MODEL_TAR_PATH.name}"
s3_model_uri = f"s3://{bucket}/{s3_key}"

s3 = boto_session.client('s3')

try:
    print(f"\n📝¤ Subiendo {MODEL_TAR_PATH.name} a S3...")
    print(f"   URI: {s3_model_uri}")
    print(f"   Esto puede tardar 1-2 minutos...")
    
    s3.upload_file(str(MODEL_TAR_PATH), bucket, s3_key)
    
    print(f"¢Å“"¦ Modelo subido y sobrescrito exitosamente")
    print(f"   URI: {s3_model_uri}")
    print("\n°Å¸"™Â¡ Ahora puedes continuar con el Paso 7")
    
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se pudo subir el modelo a S3")
    print(f"   Detalle: {e}")
    raise


Ã°Å¸""ž SOBRESCRIBIENDO MODELO EN S3...
Ã°Å¸"Â Verificando estructura del modelo local...
   Archivos en tar.gz: 4
   Tiene code/inference.py: True
   Tiene model.pth: True
   Estructura incorrecta: False
Ã¢Å“"¦ Estructura correcta - listo para subir

📝¤ Subiendo cnn1d_lstm_ecg_v1_sagemaker.tar.gz a S3...
   URI: s3://sagemaker-us-east-1-235500188303/models/cnn1d_lstm_ecg_v1/cnn1d_lstm_ecg_v1_sagemaker.tar.gz
   Esto puede tardar 1-2 minutos...
Ã¢Å“"¦ Modelo subido y sobrescrito exitosamente
   URI: s3://sagemaker-us-east-1-235500188303/models/cnn1d_lstm_ecg_v1/cnn1d_lstm_ecg_v1_sagemaker.tar.gz

Ã°Å¸"™Â¡ Ahora puedes continuar con el Paso 7


## 📝¤ Paso 6: Subir Modelo a S3

Sube el modelo comprimido a S3. Esto puede tardar 1-2 minutos dependiendo del tamaño.


In [14]:
print("=" * 70)
print("📝¤ PASO 6: SUBIENDO MODELO A S3...")
print("=" * 70)

s3_key = f"models/cnn1d_lstm_ecg_v1/{MODEL_TAR_PATH.name}"
s3_model_uri = f"s3://{bucket}/{s3_key}"

s3 = boto_session.client('s3')

try:
    # Verificar si ya existe
    try:
        s3.head_object(Bucket=bucket, Key=s3_key)
        print(f"¢Å¡Â ¯Â¸Â  El modelo ya existe en S3")
        print(f"   URI: {s3_model_uri}")
        print("   Usando modelo existente (no se sobrescribe)")
    except s3.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"   Subiendo {MODEL_TAR_PATH.name}...")
            print(f"   Esto puede tardar 1-2 minutos...")
            s3.upload_file(str(MODEL_TAR_PATH), bucket, s3_key)
            print(f"¢Å“"¦ Modelo subido exitosamente")
            print(f"   URI: {s3_model_uri}")
        else:
            raise
    
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se pudo subir el modelo a S3")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ Verifica:")
    print("   - Permisos de escritura en el bucket S3")
    print("   - Que el bucket exista")
    print("   - Que tengas conexión a internet")
    raise


📝¤ PASO 6: SUBIENDO MODELO A S3...
Ã¢Å¡Â   El modelo ya existe en S3
   URI: s3://sagemaker-us-east-1-235500188303/models/cnn1d_lstm_ecg_v1/cnn1d_lstm_ecg_v1_sagemaker.tar.gz
   Usando modelo existente (no se sobrescribe)


## °Å¸Â¤"“ Paso 7: Crear Modelo en SageMaker

Registra el modelo en SageMaker con la configuración de PyTorch.


In [20]:
from sagemaker.pytorch import PyTorchModel

print("=" * 70)
print("°Å¸Â¤"“ PASO 7: CREANDO MODELO EN SAGEMAKER...")
print("=" * 70)

try:
    model = PyTorchModel(
        model_data=s3_model_uri,
        role=role,
        framework_version='2.0.0',
        py_version='py310',
        entry_point='inference.py',
        env={
            'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
        }
    )
    print("¢Å“"¦ Modelo creado en SageMaker")
    print(f"   Framework: PyTorch 2.0.0")
    print(f"   Python: 3.10")
    print(f"   Model data: {s3_model_uri}")
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se pudo crear el modelo en SageMaker")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ Verifica:")
    print("   - Que el rol tenga permisos de SageMaker")
    print("   - Que el archivo .tar.gz tenga la estructura correcta")
    print("   - Que el modelo esté correctamente subido a S3")
    raise


Ã°Å¸Â¤"“ PASO 7: CREANDO MODELO EN SAGEMAKER...
Ã¢Å“"¦ Modelo creado en SageMaker
   Framework: PyTorch 2.0.0
   Python: 3.10
   Model data: s3://sagemaker-us-east-1-235500188303/models/cnn1d_lstm_ecg_v1/cnn1d_lstm_ecg_v1_sagemaker.tar.gz


## ¢Å¡â„¢¯Â¸Â Paso 8: Configurar Serverless Inference

Configura el modo serverless que solo cobra por invocación.


## °Å¸"Å  Ver Logs de CloudWatch

Si hay errores, revisa los logs para diagnosticar el problema.


In [36]:
# Ver logs de CloudWatch del endpoint
import boto3
from datetime import datetime, timedelta

print("=" * 70)
print("°Å¸"Å  LOGS DE CLOUDWATCH")
print("=" * 70)

logs_client = boto3.client('logs', region_name=AWS_REGION)
log_group_name = f"/aws/sagemaker/Endpoints/{ENDPOINT_NAME}"

try:
    # Obtener logs de la última hora
    end_time = int(datetime.now().timestamp() * 1000)
    start_time = int((datetime.now() - timedelta(hours=1)).timestamp() * 1000)
    
    print(f"Buscando logs en: {log_group_name}")
    print(f"Período: última hora\n")
    
    response = logs_client.filter_log_events(
        logGroupName=log_group_name,
        startTime=start_time,
        endTime=end_time,
        limit=50
    )
    
    events = response.get('events', [])
    
    if events:
        print(f"¢Å“"¦ Encontrados {len(events)} eventos de log:\n")
        for event in events[-20:]:  # Mostrar últimos 20
            timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
            message = event['message']
            print(f"[{timestamp.strftime('%H:%M:%S')}] {message}")
    else:
        print("¢Å¡Â ¯Â¸Â  No se encontraron logs recientes")
        print("\n°Å¸"™Â¡ Posibles razones:")
        print("   - El endpoint aún se está creando")
        print("   - No ha habido invocaciones recientes")
        print("   - Los logs aún no están disponibles")
        
    print(f"\n📋 Ver todos los logs en CloudWatch:")
    print(f"   https://console.aws.amazon.com/cloudwatch/home?region={AWS_REGION}#logsV2:log-groups/log-group/{log_group_name.replace('/', '$252F')}")
    
except logs_client.exceptions.ResourceNotFoundException:
    print(f"¢ÂÅ’ El log group no existe aún: {log_group_name}")
    print("   Esto es normal si el endpoint se acaba de crear")
    print("   Los logs aparecerán después de la primera invocación")
except Exception as e:
    print(f"¢ÂÅ’ Error obteniendo logs: {e}")
    print(f"\n°Å¸"™Â¡ Ver logs manualmente en:")
    print(f"   CloudWatch ¢" "™ Log groups ¢" "™ {log_group_name}")


Ã°Å¸"Å  LOGS DE CLOUDWATCH
Buscando logs en: /aws/sagemaker/Endpoints/cnn1d-lstm-ecg-v1-serverless
Período: última hora

Ã¢Å“"¦ Encontrados 50 eventos de log:

[22:55:18] 	at org.apache.logging.log4j.core.appender.RollingFileAppender$Builder.build(RollingFileAppender.java:146)
[22:55:18] 	at org.apache.logging.log4j.core.appender.RollingFileAppender$Builder.build(RollingFileAppender.java:62)
[22:55:18] 	at org.apache.logging.log4j.core.config.plugins.util.PluginBuilder.build(PluginBuilder.java:122)
[22:55:18] 	at org.apache.logging.log4j.core.config.AbstractConfiguration.createPluginObject(AbstractConfiguration.java:1120)
[22:55:18] 	at org.apache.logging.log4j.core.config.AbstractConfiguration.createConfiguration(AbstractConfiguration.java:1045)
[22:55:18] 	at org.apache.logging.log4j.core.config.AbstractConfiguration.createConfiguration(AbstractConfiguration.java:1037)
[22:55:18] 	at org.apache.logging.log4j.core.config.AbstractConfiguration.doConfigure(AbstractConfiguration.java:651

In [22]:
from sagemaker.serverless import ServerlessInferenceConfig

print("=" * 70)
print("¢Å¡â„¢¯Â¸Â  PASO 8: CONFIGURANDO SERVERLESS INFERENCE...")
print("=" * 70)

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=SERVERLESS_MEMORY_MB,
    max_concurrency=SERVERLESS_MAX_CONCURRENCY
)

print("¢Å“"¦ Configuración serverless:")
print(f"   Memoria: {SERVERLESS_MEMORY_MB} MB ({SERVERLESS_MEMORY_MB/1024:.1f} GB)")
print(f"   Max concurrencia: {SERVERLESS_MAX_CONCURRENCY}")
print(f"   Costo estimado: ~$0.00022 por inferencia")
print(f"   Sin tráfico: $0 (no hay costo cuando está inactivo)")


Ã¢Å¡â„¢  PASO 8: CONFIGURANDO SERVERLESS INFERENCE...
Ã¢Å“"¦ Configuración serverless:
   Memoria: 2048 MB (2.0 GB)
   Max concurrencia: 10
   Costo estimado: ~$0.00022 por inferencia
   Sin tráfico: $0 (no hay costo cuando está inactivo)


## 🚀 Paso 9: Desplegar Endpoint

**¢ÂÂ±¯Â¸Â IMPORTANTE:** Este paso puede tardar **5-10 minutos**. SageMaker está creando el endpoint.

No cierres el notebook mientras se despliega.


In [10]:
print("=" * 70)
print("🚀 PASO 9: DESPLEGANDO ENDPOINT SERVERLESS...")
print("=" * 70)

# Primero, verificar y eliminar endpoint/configuración existentes
print("°Å¸"Â Verificando recursos existentes...")
from botocore.exceptions import ClientError
sagemaker_client = sagemaker_session.boto_session.client('sagemaker', region_name=AWS_REGION)

# Verificar si el endpoint existe
try:
    endpoint_info = sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
    endpoint_status = endpoint_info['EndpointStatus']
    print(f"¢Å¡Â ¯Â¸Â  Endpoint existente encontrado: {ENDPOINT_NAME} (Estado: {endpoint_status})")
    
    if endpoint_status in ['InService', 'Creating', 'Updating']:
        print("°Å¸"”"˜¯Â¸Â  Eliminando endpoint existente...")
        sagemaker_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
        print("   Esperando a que se elimine (esto puede tardar 2-3 minutos)...")
        
        # Esperar a que se elimine
        import time
        waiter = sagemaker_client.get_waiter('endpoint_deleted')
        waiter.wait(EndpointName=ENDPOINT_NAME, WaiterConfig={'Delay': 10, 'MaxAttempts': 30})
        print("¢Å“"¦ Endpoint eliminado")
    elif endpoint_status in ['Deleting']:
        print("   El endpoint ya se está eliminando, esperando...")
        waiter = sagemaker_client.get_waiter('endpoint_deleted')
        waiter.wait(EndpointName=ENDPOINT_NAME, WaiterConfig={'Delay': 10, 'MaxAttempts': 30})
        print("¢Å“"¦ Endpoint eliminado")
        
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print("¢Å“"¦ No hay endpoint existente")
    else:
        raise

# Verificar si la configuración del endpoint existe
endpoint_config_name = ENDPOINT_NAME
try:
    config_info = sagemaker_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
    print(f"¢Å¡Â ¯Â¸Â  Configuración existente encontrada: {endpoint_config_name}")
    print("°Å¸"”"˜¯Â¸Â  Eliminando configuración existente...")
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    print("¢Å“"¦ Configuración eliminada")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print("¢Å“"¦ No hay configuración existente")
    else:
        raise

print("\n" + "=" * 70)
print("¢ÂÂ±¯Â¸Â  Creando nuevo endpoint...")
print("   Esto puede tardar 5-10 minutos...")
print("   (SageMaker está creando el endpoint)")
print("   No cierres el notebook mientras se despliega\n")

try:
    predictor = model.deploy(
        serverless_inference_config=serverless_config,
        endpoint_name=ENDPOINT_NAME,
        wait=True
    )
    
    print(f"\n¢Å“"¦ ENDPOINT DESPLEGADO EXITOSAMENTE!")
    print(f"   📝 Endpoint name: {predictor.endpoint_name}")
    print(f"   °Å¸""” Endpoint ARN: {predictor.endpoint}")
    
except Exception as e:
    print(f"¢ÂÅ’ ERROR: No se pudo desplegar el endpoint")
    print(f"   Detalle: {e}")
    print("\n°Å¸"™Â¡ Posibles causas:")
    print("   - El rol no tiene permisos suficientes")
    print("   - Límites de cuenta alcanzados (número de endpoints)")
    print("   - La región no soporta Serverless Inference")
    print("   - Problemas de red o conectividad")
    raise


🚀 PASO 9: DESPLEGANDO ENDPOINT SERVERLESS...
Ã°Å¸"Â Verificando recursos existentes...


NameError: name 'sagemaker_session' is not defined

## °Å¸Â§Âª Paso 10: Probar el Endpoint

Prueba que el endpoint funcione correctamente con datos de ejemplo.


In [1]:
import json
import numpy as np
import torch
from pathlib import Path

print("=" * 70)
print("°Å¸Â§Âª PASO 10: PROBANDO ENDPOINT CON ECG REAL...")
print("=" * 70)

# Cargar un ECG real desde los datos de prueba
# El modelo espera: seq_len=2000, n_channels=3
# Intentar primero con tensors_200hz (.pt), luego numpy_200hz (.npy)

# Buscar en múltiples ubicaciones posibles
base_paths = [
    Path("../data/Datos_supervisados"),  # Desde Books/
    Path("data/Datos_supervisados"),      # Desde raíz del proyecto
    Path("S:/Proyecto final/data/Datos_supervisados"),  # Ruta absoluta Windows
]

tensors_dir = None
numpy_dir = None

for base in base_paths:
    tensors_candidate = base / "tensors_200hz"
    numpy_candidate = base / "numpy_200hz"
    if tensors_candidate.exists():
        tensors_dir = tensors_candidate
    if numpy_candidate.exists():
        numpy_dir = numpy_candidate
    if tensors_dir or numpy_dir:
        break

# Si no se encontró, usar rutas por defecto
if tensors_dir is None:
    tensors_dir = Path("data/Datos_supervisados/tensors_200hz")
if numpy_dir is None:
    numpy_dir = Path("data/Datos_supervisados/numpy_200hz")

ecg_sample = None

# Intentar cargar desde tensors_200hz (archivos .pt)
if tensors_dir.exists() and (tensors_dir / "X_test.pt").exists():
    try:
        print(f"°Å¸""š Cargando ECG desde: {tensors_dir / 'X_test.pt'}")
        X_test = torch.load(tensors_dir / "X_test.pt", map_location='cpu')
        
        # Obtener el número total de ECGs disponibles
        if isinstance(X_test, torch.Tensor):
            num_ecgs = X_test.shape[0]
            X_test_np = X_test.numpy()
        else:
            X_test_np = np.array(X_test)
            num_ecgs = X_test_np.shape[0]
        
        # Seleccionar un ECG aleatorio del conjunto de prueba
        import random
        random_idx = random.randint(0, num_ecgs - 1)
        ecg_sample = X_test_np[random_idx:random_idx+1]  # [1, seq_len, n_channels]
        
        print(f"¢Å“"¦ ECG cargado exitosamente desde tensors_200hz")
        print(f"   Total de ECGs disponibles: {num_ecgs}")
        print(f"   ECG seleccionado (aleatorio): índice {random_idx}")
        print(f"   Forma: {ecg_sample.shape}")
        print(f"   Rango de valores: [{ecg_sample.min():.4f}, {ecg_sample.max():.4f}]")
        
    except Exception as e:
        print(f"¢Å¡Â ¯Â¸Â  Error cargando desde tensors_200hz: {e}")

# Si no se pudo cargar desde tensors_200hz, intentar numpy_200hz
if ecg_sample is None and numpy_dir.exists() and (numpy_dir / "X_test.npy").exists():
    try:
        print(f"°Å¸""š Cargando ECG desde: {numpy_dir / 'X_test.npy'}")
        X_test = np.load(numpy_dir / "X_test.npy", mmap_mode='r')
        
        # Obtener el número total de ECGs disponibles
        num_ecgs = X_test.shape[0]
        
        # Seleccionar un ECG aleatorio del conjunto de prueba
        import random
        random_idx = random.randint(0, num_ecgs - 1)
        ecg_sample = X_test[random_idx:random_idx+1]  # [1, seq_len, n_channels]
        
        print(f"¢Å“"¦ ECG cargado exitosamente desde numpy_200hz")
        print(f"   Total de ECGs disponibles: {num_ecgs}")
        print(f"   ECG seleccionado (aleatorio): índice {random_idx}")
        print(f"   Forma: {ecg_sample.shape}")
        print(f"   Rango de valores: [{ecg_sample.min():.4f}, {ecg_sample.max():.4f}]")
        
    except Exception as e:
        print(f"¢Å¡Â ¯Â¸Â  Error cargando desde numpy_200hz: {e}")

# Si no se pudo cargar, usar datos aleatorios
if ecg_sample is None:
    print(f"¢Å¡Â ¯Â¸Â  No se encontraron datos de ECG en:")
    print(f"   - {tensors_dir}")
    print(f"   - {numpy_dir}")
    print("   Usando datos aleatorios como fallback...")
    ecg_sample = np.random.randn(1, 2000, 3)

# Verificar y ajustar la forma si es necesario
if ecg_sample.shape[1] != 2000 or ecg_sample.shape[2] != 3:
    print(f"¢Å¡Â ¯Â¸Â  ADVERTENCIA: El ECG tiene forma {ecg_sample.shape}, pero el modelo espera [batch, 2000, 3]")
    print("   Ajustando a la forma correcta...")
    if ecg_sample.shape[1] > 2000:
        ecg_sample = ecg_sample[:, :2000, :]
    elif ecg_sample.shape[1] < 2000:
        # Padding si es necesario
        padding = np.zeros((1, 2000 - ecg_sample.shape[1], 3))
        ecg_sample = np.concatenate([ecg_sample, padding], axis=1)
    if ecg_sample.shape[2] != 3:
        if ecg_sample.shape[2] > 3:
            ecg_sample = ecg_sample[:, :, :3]
        else:
            # Padding de canales si es necesario
            padding = np.zeros((1, ecg_sample.shape[1], 3 - ecg_sample.shape[2]))
            ecg_sample = np.concatenate([ecg_sample, padding], axis=2)
    print(f"   Forma ajustada: {ecg_sample.shape}")

# Preparar datos para el endpoint
test_data = {
    "signals": ecg_sample.tolist()
}

try:
    print(f"\n📝¤ Enviando ECG al endpoint...")
    print(f"   Forma de los datos: {ecg_sample.shape}")
    
    # Usar boto3 directamente para tener control sobre la serialización JSON
    import boto3
    runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION)
    
    # Serializar JSON manualmente con ensure_ascii=False para evitar problemas de codificación
    json_body = json.dumps(test_data, ensure_ascii=False)
    print(f"   Tamaño del JSON: {len(json_body)} bytes")
    
    # Invocar endpoint
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType='application/json',
        Body=json_body.encode('utf-8')  # Codificar explícitamente a UTF-8
    )
    
    # Leer respuesta
    result = json.loads(response['Body'].read().decode('utf-8'))
    
    print("\n¢Å“"¦ Endpoint respondió correctamente!")
    print("\n   Respuesta:")
    print(json.dumps(result, indent=2))
    
    # Interpretar la respuesta
    if isinstance(result, dict):
        if 'prediction' in result:
            pred = result['prediction']
            if isinstance(pred, list) and len(pred) > 0:
                pred_value = pred[0] if isinstance(pred[0], (int, float)) else pred[0][0] if isinstance(pred[0], list) else pred
                print(f"\n°Å¸"Å  Interpretación:")
                print(f"   Predicción: {pred_value}")
                if isinstance(pred_value, (int, float)):
                    if pred_value > 0.5:
                        print(f"   Resultado: ECG ANí"MALO (probabilidad: {pred_value:.4f})")
                    else:
                        print(f"   Resultado: ECG NORMAL (probabilidad: {1-pred_value:.4f})")
    
except Exception as e:
    print(f"\n¢Å¡Â ¯Â¸Â  ADVERTENCIA: Error en la prueba del endpoint")
    print(f"   Detalle: {e}")
    print("   El endpoint está desplegado, pero hay un problema con la inferencia")
    print("   Revisa los logs en CloudWatch")
    print(f"   Logs: CloudWatch ¢" "™ Log groups ¢" "™ /aws/sagemaker/Endpoints/{ENDPOINT_NAME}")


Ã°Å¸Â§Âª PASO 10: PROBANDO ENDPOINT CON ECG REAL...
Ã°Å¸""š Cargando ECG desde: ..\data\Datos_supervisados\tensors_200hz\X_test.pt
Ã¢Å“"¦ ECG cargado exitosamente desde tensors_200hz
   Total de ECGs disponibles: 58001
   ECG seleccionado (aleatorio): índice 48767
   Forma: (1, 2000, 3)
   Rango de valores: [-0.0003, 0.9948]

📝¤ Enviando ECG al endpoint...
   Forma de los datos: (1, 2000, 3)

Ã¢Å¡Â   ADVERTENCIA: Error en la prueba del endpoint
   Detalle: name 'AWS_REGION' is not defined
   El endpoint está desplegado, pero hay un problema con la inferencia
   Revisa los logs en CloudWatch


NameError: name 'ENDPOINT_NAME' is not defined

## ¢Å“"¦ Resumen Final

Información del endpoint desplegado y cómo usarlo.


In [58]:
print("=" * 70)
print("¢Å“"¦ DESPLIEGUE COMPLETADO")
print("=" * 70)
print(f"""
📋 INFORMACIí"N DEL ENDPOINT:
   Nombre: {ENDPOINT_NAME}
   Región: {AWS_REGION}
   Tipo: Serverless Inference

💰 COSTOS:
   Por inferencia: ~$0.00022
   Sin tráfico: $0 (no hay costo cuando está inactivo)
   
   Ejemplos mensuales:
   - 1,000 inferencias:    $0.22
   - 10,000 inferencias:   $2.20
   - 100,000 inferencias:  $22.00

📝 Cí"MO USAR EL ENDPOINT:

   # Desde Python:
   import boto3
   import json
   import numpy as np
   
   runtime = boto3.client('sagemaker-runtime', region_name='{AWS_REGION}')
   
   data = {{
       "signals": np.random.randn(1, 2000, 3).tolist()
   }}
   
   response = runtime.invoke_endpoint(
       EndpointName='{ENDPOINT_NAME}',
       ContentType='application/json',
       Body=json.dumps(data)
   )
   
   result = json.loads(response['Body'].read())
   print(result)

°Å¸"”"˜¯Â¸Â  PARA ELIMINAR EL ENDPOINT (cuando no lo uses):
   
   predictor.delete_endpoint()
   predictor.delete_model()
   
   O desde la consola AWS:
   - SageMaker ¢" "™ Endpoints ¢" "™ {ENDPOINT_NAME} ¢" "™ Delete

°Å¸"Å  MONITOREO:
   - Logs: CloudWatch ¢" "™ Log groups ¢" "™ /aws/sagemaker/Endpoints/{ENDPOINT_NAME}
   - Métricas: SageMaker ¢" "™ Endpoints ¢" "™ {ENDPOINT_NAME} ¢" "™ Monitoring
""")

print("\n" + "=" * 70)
print("°Å¸Å½"° ‚Â¡Todo listo! Tu modelo está desplegado y funcionando.")
print("=" * 70)


Ã¢Å“"¦ DESPLIEGUE COMPLETADO

📋 INFORMACIÃƒ"N DEL ENDPOINT:
   Nombre: cnn1d-lstm-ecg-v1-serverless
   Región: us-east-1
   Tipo: Serverless Inference

💰 COSTOS:
   Por inferencia: ~$0.00022
   Sin tráfico: $0 (no hay costo cuando está inactivo)

   Ejemplos mensuales:
   - 1,000 inferencias:    $0.22
   - 10,000 inferencias:   $2.20
   - 100,000 inferencias:  $22.00

📝 CÃƒ"MO USAR EL ENDPOINT:

   # Desde Python:
   import boto3
   import json
   import numpy as np

   runtime = boto3.client('sagemaker-runtime', region_name='us-east-1')

   data = {
       "signals": np.random.randn(1, 2000, 3).tolist()
   }

   response = runtime.invoke_endpoint(
       EndpointName='cnn1d-lstm-ecg-v1-serverless',
       ContentType='application/json',
       Body=json.dumps(data)
   )

   result = json.loads(response['Body'].read())
   print(result)

Ã°Å¸"”"˜  PARA ELIMINAR EL ENDPOINT (cuando no lo uses):

   predictor.delete_endpoint()
   predictor.delete_model()

   O desde la consola AWS:
   -

## °Å¸"”"˜¯Â¸Â Eliminar Endpoint (Opcional)

Ejecuta esta celda solo si quieres eliminar el endpoint y el modelo para evitar costos.


In [ ]:
# ¢Å¡Â ¯Â¸Â ADVERTENCIA: Esto eliminará el endpoint y el modelo
# Solo ejecuta esta celda si estás seguro de que quieres eliminar todo

# Descomenta las siguientes líneas para eliminar:

# predictor.delete_endpoint()
# predictor.delete_model()
# print("¢Å“"¦ Endpoint y modelo eliminados")

print("¢Å¡Â ¯Â¸Â  Esta celda está deshabilitada por seguridad.")
print("   Para eliminar el endpoint, descomenta las líneas anteriores.")
